In [ ]:
data_dir='D:/food-5'
categories=os.listdir(data_dir)
nb_classes=len(categories)
image_w=128
image_h=128
pixels=image_h*image_w*3

X=[]
y=[]

In [ ]:
for idx,cat in enumerate(categories):
    #one-hot
    label=[0 for i in range(nb_classes)]
    label[idx]=1
    
    image_dir=data_dir+ '/' +cat
    files=glob.glob(image_dir+'*.jpg')
    print(cat,'파일길이:', len(files))
    
    for i, f in enumerate(files):
        img=image.open(f)
        img=img.convert('RGB')
        img=img.resize((image_w,image_h))
        data=np.asarray(img)
        
        X.append(data)
        y.append(label)
        
        print(cat,":",f)
        
X=np.array(X)
y=np.array(y)

X_train,X_test,y_train,y_test=train_test_split(X,y)
xy=(X_train,X_test,y_train,y_test)
np.save('./food8.npy',xy)

print('ok',len(y))
        

In [ ]:
#모델
import os,glob,numpy as np
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten, Dorpout
from keras.callbacks import EarlyStopping, ModelChekpoint
import tensorflow as tf
import matplotlib.pyplot as plt
import keras.backend.tensorflow_backend as K
import pathlib

from keras.wrwppers.scikit_learn import KerasClassfier
from sklearn.model_selection import GridSearchCV

config=tf.ConfigProto()
config.gpu_options.allos_growth=True
session=tf.Session(config=config)

X_train,X_test,y_train,y_test=np.load('./food8.npy')


In [ ]:
data_dir='D:/food-5'
data_dir=pathlib.Path(data_dir)
categories=os.listdir(data_dir)
nb_classes=len(categories)

In [ ]:
X_train=X_train.astype(float)/255
X_test=X_test.astype(float)/255


In [ ]:
X_train.shape

In [ ]:
categories

In [ ]:
plt.imshow(X_train[0])

In [ ]:
y_train[0]

In [ ]:
model=Sequentail()
#input layer
model.add(Conv2D(32,(3,3),padding='same',
                input_shape=X_train.shape[1:],
                activation='relu',
                init='he_uniform'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
#hidden layer1
model.add(Conv2D(64,(3,3),padding='same',
                activation='relu',
                init='he_uniform'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#hidden layer2
model.add(Conv2D(128,(3,3),padding='same',
                activation='relu',
                init='he_uniform'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#flatten output layer
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))
model.compile(loss='categorical_crossnetropy',optimizer='adam',metrics=['accuracy'])
model_dir='./model'

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

model_path=model_dir+ '/muti_img_clssification8.model'
checkpoint=ModelChekpoint(filepath=model_path,monitor='val_loss',verbose=1,save_best_onlly=True)
early_stopping=EarlyStopping(monitor='val_loss',patience=10)


In [ ]:
history=model.fit(X_train,y_train,batch_size=20,epochs=50,
                 validation_data=(X_test,y_test),
                 callbacks=[checkpoint,early_stopping])

In [ ]:
print('정확도: %.4f' % (model.evaluate(X_test,y_test)[1]))

In [ ]:
y_vloss=history.history['val_loss']
y_loss=history.history['loss']

x_len=np.arange(len(y_loss))
plt.plot(x_len,y_vloss,marker='.',c='red',label='val_set_loss')
plt.plot(x_len,y_loss,marker='.',c='blue',label='train_set_loss')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('loss')
plt.grid()
plt.show()